https://pytorch3d.org/tutorials/deform_source_mesh_to_target_mesh


In [ ]:
import os
import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)
import numpy as np
from tqdm import tqdm

# %matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["savefig.dpi"] = 80
mpl.rcParams["figure.dpi"] = 80

from plotly.subplots import make_subplots

from lac.utils.plotting import plot_3d_points

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

# Mesh deformation


## Load mesh


In [ ]:
!wget https://dl.fbaipublicfiles.com/pytorch3d/data/dolphin/dolphin.obj

In [ ]:
# Load the dolphin mesh.
trg_obj = "dolphin.obj"

In [ ]:
# We read the target 3D model using load_obj
verts, faces, aux = load_obj(trg_obj)

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We scale normalize and center the target mesh to fit in a sphere of radius 1 centered at (0,0,0).
# (scale, center) will be used to bring the predicted mesh to its original center and scale
# Note that normalizing the target mesh, speeds up the optimization but is not necessary!
center = verts.mean(0)
verts = verts - center
scale = max(verts.abs().max(0)[0])
verts = verts / scale

# We construct a Meshes structure for the target mesh
trg_mesh = Meshes(verts=[verts], faces=[faces_idx])

In [ ]:
# We initialize the source shape to be a sphere of radius 1
src_mesh = ico_sphere(4, device)

## Visualize


In [ ]:
def plot_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    points = points.clone().detach().cpu().squeeze()
    fig = plot_3d_points(points)
    return fig

In [ ]:
def plot_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [ ]:
plot_pointcloud(trg_mesh)

In [ ]:
plot_pointcloud(src_mesh)

In [ ]:
# fig = make_subplots(
#     rows=1, cols=2,
#     specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]],
#     subplot_titles=["3D Scatter", "3D Line"]
# )
# for trace in plot_pointcloud(trg_mesh).data:
#     fig.add_trace(trace, row=1, col=1)
# for trace in plot_pointcloud(src_mesh).data:
#     fig.add_trace(trace, row=1, col=2)
# fig.show()

## Optimization


In [ ]:
# We will learn to deform the source mesh by offsetting its vertices
# The shape of the deform parameters is equal to the total number of vertices in src_mesh
deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)

In [ ]:
# The optimizer
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)

In [ ]:
# Number of optimization steps
Niter = 2000
# Weight for the chamfer loss
w_chamfer = 1.0
# Weight for mesh edge loss
w_edge = 1.0
# Weight for mesh normal consistency
w_normal = 0.01
# Weight for mesh laplacian smoothing
w_laplacian = 0.1
# Plot period for the losses
plot_period = 250
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []

# %matplotlib inline

for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = (
        loss_chamfer * w_chamfer
        + loss_edge * w_edge
        + loss_normal * w_normal
        + loss_laplacian * w_laplacian
    )

    # Print the losses
    loop.set_description("total_loss = %.6f" % loss)

    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    edge_losses.append(float(loss_edge.detach().cpu()))
    normal_losses.append(float(loss_normal.detach().cpu()))
    laplacian_losses.append(float(loss_laplacian.detach().cpu()))

    # Plot mesh
    if i % plot_period == 0:
        plot_pointcloud(new_src_mesh, title="iter: %d" % i)

    # Optimization step
    loss.backward()
    optimizer.step()

In [ ]:
fig = plt.figure(figsize=(13, 5))
ax = fig.gca()
ax.plot(chamfer_losses, label="chamfer loss")
ax.plot(edge_losses, label="edge loss")
ax.plot(normal_losses, label="normal loss")
ax.plot(laplacian_losses, label="laplacian loss")
ax.legend(fontsize="16")
ax.set_xlabel("Iteration", fontsize="16")
ax.set_ylabel("Loss", fontsize="16")
ax.set_title("Loss vs iterations", fontsize="16")

In [ ]:
# Fetch the verts and faces of the final predicted mesh
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

# Scale normalize back to the original target size
final_verts = final_verts * scale + center

# Store the predicted mesh using save_obj
final_obj = "final_model.obj"
save_obj(final_obj, final_verts, final_faces)

# Terrain

Try deforming a plane mesh to a target heightmap


In [ ]:
from pytorch3d_utils import structured_grid_to_pytorch3d_mesh

In [ ]:
map = np.load("../../../data/heightmaps/qualifying/Moon_Map_01_preset_0.dat", allow_pickle=True)

trg_mesh = structured_grid_to_pytorch3d_mesh(map[..., :3])
trg_mesh = trg_mesh.to(device)

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection


def plot_mesh_vertices_edges(mesh, title="Mesh Visualization"):
    """
    Plots the vertices and edges of a PyTorch3D mesh using Matplotlib.

    Args:
        mesh: A PyTorch3D Meshes object.
        title: Title of the plot.
    """
    verts = mesh.verts_packed().clone().detach().cpu().numpy()  # Extract vertex positions
    faces = mesh.faces_packed().clone().detach().cpu().numpy()  # Extract face indices

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")

    # Plot vertices
    ax.scatter(verts[:, 0], verts[:, 1], verts[:, 2], color="red", s=10, label="Vertices")

    # Create edges from face indices
    edges = set()
    for face in faces:
        for i in range(3):
            edge = tuple(sorted((face[i], face[(i + 1) % 3])))  # Ensure unique edges
            edges.add(edge)

    edge_lines = [[verts[i], verts[j]] for i, j in edges]

    # Plot edges
    ax.add_collection3d(Line3DCollection(edge_lines, colors="black", linewidths=0.5))

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    # ax.set_title(title)
    # plt.axis("equal")
    plt.show()
    return fig

In [ ]:
plot_mesh_vertices_edges(trg_mesh)

In [ ]:
MAP_EXTENT = 4.425
MAP_RES = 0.15
N = 60

# Initialize src_mesh as plane with same number of vertices as trg_mesh
x = np.linspace(-MAP_EXTENT, MAP_EXTENT, 60)
y = np.linspace(-MAP_EXTENT, MAP_EXTENT, 60)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
plane_grid = np.stack([X, Y, Z], axis=-1)
src_mesh = structured_grid_to_pytorch3d_mesh(plane_grid)
src_mesh = src_mesh.to(device)

In [ ]:
# def plot_pointcloud(mesh, title=""):
#     # Sample points uniformly from the surface of the mesh.
#     points = sample_points_from_meshes(mesh, 5000)
#     points = points.clone().detach().cpu().squeeze()
#     fig = plot_3d_points(points)
#     return fig


def plot_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, y, z, s=2)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    # ax.view_init(190, 30)
    plt.axis("equal")
    plt.show()

In [ ]:
# We will learn to deform the source mesh by offsetting its vertices
# The shape of the deform parameters is equal to the total number of vertices in src_mesh
deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)

# The optimizer
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)

In [ ]:
import imageio

filenames = []

In [ ]:
# Number of optimization steps
Niter = 10000
# Weight for the chamfer loss
w_chamfer = 1.0
# Weight for mesh edge loss
w_edge = 1.0
# Weight for mesh normal consistency
w_normal = 0.01
# Weight for mesh laplacian smoothing
w_laplacian = 0.1
# Plot period for the losses
plot_period = 250
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []

# %matplotlib inline

for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = (
        loss_chamfer * w_chamfer
        + loss_edge * w_edge
        + loss_normal * w_normal
        + loss_laplacian * w_laplacian
    )

    # Print the losses
    loop.set_description("total_loss = %.6f" % loss)

    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    edge_losses.append(float(loss_edge.detach().cpu()))
    normal_losses.append(float(loss_normal.detach().cpu()))
    laplacian_losses.append(float(loss_laplacian.detach().cpu()))

    # Plot mesh
    if i % plot_period == 0:
        # plot_pointcloud(new_src_mesh, title="iter: %d" % i)
        fig = plot_mesh_vertices_edges(new_src_mesh)
        filename = f"frames/frame_{i}.png"
        filenames.append(filename)
        fig.savefig(filename)
        plt.close(fig)

    # Optimization step
    loss.backward()
    optimizer.step()

In [ ]:
imageio.mimsave("animation.gif", [imageio.imread(f) for f in filenames], duration=0.1)

In [ ]:
fig = plt.figure(figsize=(13, 5))
ax = fig.gca()
ax.plot(chamfer_losses, label="chamfer loss")
ax.plot(edge_losses, label="edge loss")
ax.plot(normal_losses, label="normal loss")
ax.plot(laplacian_losses, label="laplacian loss")
ax.legend(fontsize="16")
ax.set_xlabel("Iteration", fontsize="16")
ax.set_ylabel("Loss", fontsize="16")
ax.set_title("Loss vs iterations", fontsize="16")

In [ ]:
# Fetch the verts and faces of the final predicted mesh
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

# Scale normalize back to the original target size
final_verts = final_verts * scale + center

# Store the predicted mesh using save_obj
final_obj = "final_model.obj"
save_obj(final_obj, final_verts, final_faces)

In [ ]:
new_src_mesh.verts